In [1]:
import sys
sys.path.append("../")

from DiffusionModules.Diffusion import *
from DiffusionModules.DiffusionTrainer import *
from DiffusionModules.DiffusionModels import *
from DiffusionModules.LatentVQGANModules import *
import os
import torch
import torchvision
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from torchinfo import summary
plt.rcParams['figure.figsize'] = [10, 5]

2023-06-17 13:40:11.074645: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-17 13:40:11.919801: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-17 13:40:11.919924: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-17 13:40:11.919937: W tensorfl

In [2]:
encoder = Encoder(
    ch=256,
    in_channels=3,
    resolution=256,
    z_channels=256,
    num_res_blocks=2,
    attn_resolutions=[64, 32],
    ch_mult=(1,2,4,8),
    dropout=0.1,
)

summary(encoder, (1, 3, 256, 256), verbose=1)
0

Layer (type:depth-idx)                        Output Shape              Param #
Encoder                                       [1, 512, 32, 32]          --
├─Conv2d: 1-1                                 [1, 256, 256, 256]        7,168
├─ModuleList: 1-2                             --                        --
│    └─MultiParamSequential: 2-1              [1, 256, 128, 128]        --
│    │    └─ResBlock: 3-1                     [1, 256, 256, 256]        1,771,264
│    │    └─ResBlock: 3-2                     [1, 256, 128, 128]        1,771,264
│    └─MultiParamSequential: 2-2              [1, 512, 64, 64]          --
│    │    └─ResBlock: 3-3                     [1, 512, 128, 128]        4,721,664
│    │    └─ResBlock: 3-4                     [1, 512, 64, 64]          7,081,472
│    └─MultiParamSequential: 2-3              [1, 1024, 32, 32]         --
│    │    └─SelfAttentionResBlock: 3-5        [1, 1024, 64, 64]         23,080,960
│    │    └─ResBlock: 3-6                     [1, 1024, 

0

In [3]:
encoder_emb = Encoder(
    ch=256,
    in_channels=3,
    resolution=256,
    z_channels=256,
    num_res_blocks=2,
    attn_resolutions=[64, 32],
    ch_mult=(1,2,4,8),
    dropout=0.1,
    emb_size=512,
    out_emb_size=1024
)

summary(encoder_emb, [(1, 3, 256, 256), (1, 512)], verbose=1)
0

Layer (type:depth-idx)                        Output Shape              Param #
Encoder                                       [1, 512, 32, 32]          --
├─Sequential: 1-1                             [1, 1024]                 --
│    └─Linear: 2-1                            [1, 1024]                 525,312
│    └─SiLU: 2-2                              [1, 1024]                 --
│    └─Linear: 2-3                            [1, 1024]                 1,049,600
├─Conv2d: 1-2                                 [1, 256, 256, 256]        7,168
├─ModuleList: 1-3                             --                        --
│    └─MultiParamSequential: 2-4              [1, 256, 128, 128]        --
│    │    └─ResBlock: 3-1                     [1, 256, 256, 256]        2,296,064
│    │    └─ResBlock: 3-2                     [1, 256, 128, 128]        2,296,064
│    └─MultiParamSequential: 2-5              [1, 512, 64, 64]          --
│    │    └─ResBlock: 3-3                     [1, 512, 128, 128]  

0

In [4]:
decoder = Decoder(
    ch=256,
    out_ch=3,
    resolution=256,
    z_channels=256,
    num_res_blocks=2,
    attn_resolutions=[64, 32],
    ch_mult=(1,2,4,8),
    dropout=0.1,
)

summary(decoder, (1, 256, 32, 32), verbose=1)
0

Layer (type:depth-idx)                        Output Shape              Param #
Decoder                                       [1, 3, 256, 256]          --
├─Conv2d: 1-1                                 [1, 2048, 32, 32]         4,720,640
├─MultiParamSequential: 1-2                   [1, 2048, 32, 32]         --
│    └─SelfAttentionResBlock: 2-1             [1, 2048, 32, 32]         --
│    │    └─Sequential: 3-1                   [1, 2048, 32, 32]         4,096
│    │    └─Identity: 3-2                     [1, 2048, 32, 32]         --
│    │    └─Conv2d: 3-3                       [1, 2048, 32, 32]         37,750,784
│    │    └─GroupNorm: 3-4                    [1, 2048, 32, 32]         4,096
│    │    └─Sequential: 3-5                   [1, 2048, 32, 32]         37,750,784
│    │    └─Identity: 3-6                     [1, 2048, 32, 32]         --
│    │    └─Conv2d: 3-7                       [1, 2048, 32, 32]         37,750,784
│    │    └─SelfAttention: 3-8                [1, 2048, 32

0

In [5]:
decoder_emb = Decoder(
    ch=256,
    out_ch=3,
    resolution=256,
    z_channels=256,
    num_res_blocks=2,
    attn_resolutions=[64, 32],
    ch_mult=(1,2,4,8),
    dropout=0.1,
    emb_size=512,
    out_emb_size=1024
)

summary(decoder_emb, [(1, 256, 32, 32), (1, 512)], verbose=1)
0

Layer (type:depth-idx)                        Output Shape              Param #
Decoder                                       [1, 3, 256, 256]          --
├─Sequential: 1-1                             [1, 1024]                 --
│    └─Linear: 2-1                            [1, 1024]                 525,312
│    └─SiLU: 2-2                              [1, 1024]                 --
│    └─Linear: 2-3                            [1, 1024]                 1,049,600
├─Conv2d: 1-2                                 [1, 2048, 32, 32]         4,720,640
├─MultiParamSequential: 1-3                   [1, 2048, 32, 32]         --
│    └─SelfAttentionResBlock: 2-4             [1, 2048, 32, 32]         --
│    │    └─Sequential: 3-1                   [1, 2048, 32, 32]         4,096
│    │    └─Identity: 3-2                     [1, 2048, 32, 32]         --
│    │    └─Conv2d: 3-3                       [1, 2048, 32, 32]         37,750,784
│    │    └─Sequential: 3-4                   [1, 4096]          

0

In [6]:
disc = NLayerDiscriminator()

summary(disc, (1, 3, 256, 256), verbose=1)
0

Layer (type:depth-idx)                   Output Shape              Param #
NLayerDiscriminator                      [1, 1, 30, 30]            --
├─Sequential: 1-1                        [1, 1, 30, 30]            --
│    └─Conv2d: 2-1                       [1, 64, 128, 128]         3,136
│    └─LeakyReLU: 2-2                    [1, 64, 128, 128]         --
│    └─Conv2d: 2-3                       [1, 128, 64, 64]          131,072
│    └─BatchNorm2d: 2-4                  [1, 128, 64, 64]          256
│    └─LeakyReLU: 2-5                    [1, 128, 64, 64]          --
│    └─Conv2d: 2-6                       [1, 256, 32, 32]          524,288
│    └─BatchNorm2d: 2-7                  [1, 256, 32, 32]          512
│    └─LeakyReLU: 2-8                    [1, 256, 32, 32]          --
│    └─Conv2d: 2-9                       [1, 512, 31, 31]          2,097,152
│    └─BatchNorm2d: 2-10                 [1, 512, 31, 31]          1,024
│    └─LeakyReLU: 2-11                   [1, 512, 31, 31]   

0

In [7]:
disc_res = ResidualEmbeddingConditionalDiscriminator(
    in_channels=3,
    input_resolution=256,
    number_hidden_layers=4,
    base_channels=32
)

summary(disc_res, (1, 3, 256, 256), verbose=1)
0

Layer (type:depth-idx)                        Output Shape              Param #
ResidualEmbeddingConditionalDiscriminator     [1, 1, 32, 32]            --
├─Sequential: 1-1                             [1, 32, 256, 256]         --
│    └─Conv2d: 2-1                            [1, 32, 256, 256]         896
│    └─SiLU: 2-2                              [1, 32, 256, 256]         --
├─MultiParamSequential: 1-2                   [1, 256, 32, 32]          --
│    └─ResBlock: 2-3                          [1, 64, 128, 128]         --
│    │    └─Sequential: 3-1                   [1, 32, 256, 256]         64
│    │    └─AvgPool2d: 3-2                    [1, 32, 128, 128]         --
│    │    └─Conv2d: 3-3                       [1, 64, 128, 128]         18,496
│    │    └─GroupNorm: 3-4                    [1, 64, 128, 128]         128
│    │    └─Sequential: 3-5                   [1, 64, 128, 128]         36,928
│    │    └─AvgPool2d: 3-6                    [1, 32, 128, 128]         --
│    │    

0

In [8]:
disc_res_emb = ResidualEmbeddingConditionalDiscriminator(
    in_channels=3,
    input_resolution=256,
    emb_size=512,
    out_emb_size=1024,
    number_hidden_layers=4,
    base_channels=32
)

summary(disc_res_emb, [(1, 3, 256, 256), (1, 512)], verbose=1)
0

Layer (type:depth-idx)                        Output Shape              Param #
ResidualEmbeddingConditionalDiscriminator     [1, 1, 32, 32]            --
├─Sequential: 1-1                             [1, 1024]                 --
│    └─Linear: 2-1                            [1, 1024]                 525,312
│    └─SiLU: 2-2                              [1, 1024]                 --
│    └─Linear: 2-3                            [1, 1024]                 1,049,600
├─Sequential: 1-2                             [1, 32, 256, 256]         --
│    └─Conv2d: 2-4                            [1, 32, 256, 256]         896
│    └─SiLU: 2-5                              [1, 32, 256, 256]         --
├─MultiParamSequential: 1-3                   [1, 256, 32, 32]          --
│    └─ResBlock: 2-6                          [1, 64, 128, 128]         --
│    │    └─Sequential: 3-1                   [1, 32, 256, 256]         64
│    │    └─AvgPool2d: 3-2                    [1, 32, 128, 128]         --
│    │ 

0

In [9]:
disc_res_emb_no_patch = ResidualEmbeddingConditionalDiscriminator(
    in_channels=3,
    input_resolution=256,
    emb_size=512,
    out_emb_size=1024,
    number_hidden_layers=4,
    patch_based=False,
    base_channels=32
)

summary(disc_res_emb_no_patch, [(1, 3, 256, 256), (1, 512)], verbose=1)
0

Layer (type:depth-idx)                        Output Shape              Param #
ResidualEmbeddingConditionalDiscriminator     [1, 1]                    --
├─Sequential: 1-1                             [1, 1024]                 --
│    └─Linear: 2-1                            [1, 1024]                 525,312
│    └─SiLU: 2-2                              [1, 1024]                 --
│    └─Linear: 2-3                            [1, 1024]                 1,049,600
├─Sequential: 1-2                             [1, 32, 256, 256]         --
│    └─Conv2d: 2-4                            [1, 32, 256, 256]         896
│    └─SiLU: 2-5                              [1, 32, 256, 256]         --
├─MultiParamSequential: 1-3                   [1, 256, 32, 32]          --
│    └─ResBlock: 2-6                          [1, 64, 128, 128]         --
│    │    └─Sequential: 3-1                   [1, 32, 256, 256]         64
│    │    └─AvgPool2d: 3-2                    [1, 32, 128, 128]         --
│    │ 

0